# Carga de datos 


## Librerias 

In [1]:

# Importación de las funciones creadas en nuestro archivo de soporte
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_queries_creacion as sqc


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd
# Para trabajar con postgresql
# -----------------------------------------------------------------------
import psycopg2

# Para trabajar gestionar los nulos
# -----------------------------------------------------------------------
import numpy as np

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

## Creacion de base de datos

In [2]:
def establecer_conn(database_name, postgres_pass, usuario, host="localhost"):
    """
    Establece una conexión a una base de datos de PostgreSQL.

    Params:
        - database_name (str): El nombre de la base de datos a la que conectarse.
        - postgres_pass (str): La contraseña del usuario de PostgreSQL.
        - usuario (str): El nombre del usuario de PostgreSQL.
        - host (str, opcional): La dirección del servidor PostgreSQL. Por defecto es "localhost".

    Returns:
        psycopg2.extensions.connection: La conexión establecida a la base de datos PostgreSQL.

    """

    # Crear la conexión a la base de datos PostgreSQL
    conn = psycopg2.connect(
        host=host,
        user=usuario,
        password=postgres_pass,
        database=database_name
    )

    # Establecer la conexión en modo autocommit
    conn.autocommit = True # No hace necesario el uso del commit al final de cada sentencia de insert, delete, etc.
    
    return conn



# conexion a postgres
conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos

# creamos un cursor con la conexion que acabamos de crear
cur = conn.cursor()


In [3]:
def crear_db(database_name):
    # conexion a postgres
    conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos
    
    # creamos un cursor con la conexion que acabamos de crear
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (database_name,))
    
    # Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
    bbdd_existe = cur.fetchone()
    
    # Si bbdd_existe es None, creamos la base de datos
    if not bbdd_existe:
        cur.execute(f"CREATE DATABASE {database_name};")
        print(f"Base de datos {database_name} creada con éxito")
    else:
        print(f"La base de datos ya existe")
        
    # Cerramos el cursor y la conexion
    cur.close()
    conn.close()


In [4]:
crear_db("miniproyecto")

La base de datos ya existe


## Creacion de base de Tablas


In [ ]:
# establecemos la conexión a base de datos creada en el paso anterior
conexion = establecer_conn("miniproyecto", "admin", "postgres")

# creamos cada una de las tablas de la base de datos.
conexion.cursor().execute(sqc.query_creation_hospitales)
conexion.cursor().execute(sqc.query_creation_tipo_hosp)
conexion.cursor().execute(sqc.query_creation_gastos)
conexion.cursor().execute(sqc.query_creation_ingresos)

In [8]:
df_gastos = pd.read_csv("../datos/raw/gastos.csv")
df_ingresos = pd.read_csv("../datos/raw/ingresos.csv")

In [12]:
df_gastos.dtypes

año                    int64
ncodi                float64
totalcompra            int64
producfarma          float64
materialsani           int64
implantes            float64
restomateriasani     float64
servcontratado       float64
trabajocontratado    float64
xrestocompras        float64
variaexistencias     float64
servexteriores         int64
sumistro             float64
xrestoserviexter     float64
gastopersonal          int64
sueldos                int64
indemnizacion        float64
segsocempresa          int64
otrgassocial         float64
dotaamortizacion     float64
perdidadeterioro     float64
xrestogasto          float64
totcompragasto         int64
dtype: object

In [20]:
df_hospitales = pd.DataFrame()
df_hospitales["ncodi"] = df_ingresos["ncodi"]
df_hospitales["nombre"] = "Unknown"
df_hospitales

,ncodi,nombre
0,"990,00",Unknown
1,"680,00",Unknown
2,"411,00",Unknown
3,"241,00",Unknown
4,"558,00",Unknown
...,...,...
55321,970,Unknown
55322,1019,Unknown
55323,1016,Unknown
55324,1047,Unknown


In [27]:
df_tipo_hospitalizacion = df_ingresos["tipo"].reset_index()
df_tipo_hospitalizacion["nombre"] = "Unknown"
df_tipo_hospitalizacion.drop(columns=["index"], inplace=True)
df_tipo_hospitalizacion

,tipo,nombre
0,Hospital,Unknown
1,Hospital,Unknown
2,Hospital,Unknown
3,Hospital,Unknown
4,Hospital,Unknown
...,...,...
55321,hospDom,Unknown
55322,hospDom,Unknown
55323,hospDom,Unknown
55324,hospDom,Unknown


In [39]:
df_gastos_limpio = df_gastos
lista_orden = ["anio",
        "ncodi",
        "total_compra",
        "productos_farmaceuticos",
        "material_sanitario",
        "implantes",
        "resto_material_sanitario",
        "servicios_contratados",
        "trabajos_contratados",
        "resto_compras",
        "resto_servicios_externos",
        "gasto_personal",
        "sueldos",
        "indemnizacion",
        "seguridad_social_empresa",
        "otros_gastos_sociales",
        "dotacion_amortizacion",
        "perdida_deterioro",
        "resto_gasto",
        "total_compra_gasto"]
df_gastos_limpio.reindex(columns=lista_orden)
df_gastos_limpio

,año,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,variaexistencias,servexteriores,sumistro,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto
0,2020,113.0,1535894,344330.0,169443,0.0,169443.0,1877.0,678201.0,342043.0,3039.0,719835,213817.0,506018.0,26209836,19875799,349499.0,5984538,0.0,298707.0,-6504.0,50165.0,28810972
1,2020,619.0,7884157,971762.0,1749006,493444.0,1255562.0,3800207.0,1248674.0,114508.0,24083.0,2844980,209305.0,2635675.0,7083969,5203283,151380.0,1591074,138232.0,2423752.0,0.0,312984.0,20573925
2,2020,163.0,5453290,126493.0,1412366,574127.0,838239.0,3217684.0,647852.0,48895.0,-26827.0,979733,97459.0,882274.0,2890388,2180177,-402.0,663398,47215.0,202688.0,-54028.0,232471.0,9677715
3,2020,131.0,245802,72318.0,52506,0.0,52506.0,1114.0,24303.0,95561.0,0.0,566931,79878.0,487053.0,1887375,1437421,9506.0,416398,24050.0,12084.0,240.0,132044.0,2844476
4,2020,375.0,762218,114702.0,106844,60.0,106784.0,14526.0,188463.0,337683.0,242.0,231222,39090.0,192132.0,7764302,5838470,0.0,1837175,88657.0,184413.0,93708.0,1552.0,9037657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8445,2012,970.0,271324,8510.0,166422,0.0,166422.0,0.0,96392.0,0.0,0.0,495914,405162.0,90752.0,1010376,796935,0.0,213441,0.0,104331.0,0.0,63796.0,1945741
8446,2012,1019.0,1687968,126665.0,94290,0.0,94290.0,1080335.0,300411.0,86267.0,-3612.0,228938,174167.0,54771.0,2447989,1852771,0.0,590887,4331.0,132364.0,0.0,0.0,4493647
8447,2012,1016.0,5418532,336983.0,1152639,145404.0,1007235.0,2739305.0,1126019.0,63586.0,0.0,1791212,430543.0,1360669.0,2624854,2066623,0.0,545427,12804.0,32691.0,99382.0,199998.0,10166669
8448,2012,1047.0,15537164,5516672.0,5255744,565150.0,4690594.0,0.0,3287539.0,1477209.0,-320553.0,7269413,2460724.0,4808689.0,43164526,34899381,0.0,8056716,208429.0,3297451.0,0.0,0.0,68948001


In [32]:
df_ingresos_limpio = df_ingresos.drop(columns=["ncodi", "tipo", "Unnamed: 0"])
df_ingresos_limpio

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,año
0,0.0,103677.0,64383.0,39294.0,61456.0,2013
1,324687.0,478280.0,392084.0,86196.0,484228.0,2013
2,793512.0,1803352.0,1803352.0,0.0,20402.0,2013
3,0.0,0.0,0.0,0.0,0.0,2013
4,24406.0,63557.0,55091.0,8466.0,44106.0,2013
...,...,...,...,...,...,...
55321,0.0,0.0,0.0,0.0,0.0,2016
55322,0.0,0.0,0.0,0.0,0.0,2016
55323,0.0,0.0,0.0,0.0,0.0,2016
55324,0.0,0.0,0.0,0.0,0.0,2016
